In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib qt5

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

In [ ]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.readdata import *

In [ ]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [ ]:
data_date = "2019_07_12_14_21_00"
data_dir = f"/home/pleroy/DATA/PoSAR-X/PIMA-1/2019_07_12/{data_date}"

In [ ]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarXParameters( params_filename )
params.print()

In [ ]:
buffersPerFile = params.buffersPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "buffersPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    buffersPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 0
nbFiles = 64
lastFile = firstFile + nbFiles - 1
firstBuffer = firstFile * buffersPerFile

# Read the data

In [ ]:
scalingFactor = 2 / 65535 
offset = -32768

In [ ]:
timeSerie_A = np.zeros( samplesPerFile )
adc_A = np.zeros( (nbFiles, samplesPerFile) )

In [ ]:
lastBuffer = (nbFiles-1) * buffersPerFile + firstBuffer
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * buffersPerFile + firstBuffer) )
    filename = data_dir + "/record" + nb + ".bin"
    readFileADLINKCh0( filename, samplesPerFile, timeSerie_A )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBuffer))
    
    adc_A[ k, : ] = (timeSerie_A + offset) * scalingFactor

In [ ]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)

## Spectrum

### No window

In [ ]:
samplesPerDownRamp = int(samplesPerRamp/2)

In [ ]:
fft_A = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp], axis=1)

In [ ]:
fft_abs = 20 * np.log10( np.abs( fft_A ) )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.imshow( fft_abs[::100, 500:2000] )
title = "no window " + data_date
ax.set_title(title)
fig.savefig( f"{data_dir}/{title}.png" )

### Hanning

In [ ]:
hanning = np.hanning(samplesPerDownRamp)

In [ ]:
fft_A_hanning = np.fft.ifft(A_reshaped[:, 0:samplesPerDownRamp]*hanning, axis=1)

In [ ]:
fft_abs_hanning = 20 * np.log10( np.abs( fft_A_hanning ) )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.imshow( fft_abs_hanning[::100, 500:2000] )
title = "hanning " + data_date
ax.set_title(title)
fig.savefig( f"{data_dir}/{title}.png" )

### Compare data

In [ ]:
idxLine = 0
fig, ax = plt.subplots(1,1)
ax.plot( fft_abs[idxLine,:], label="raw"  )
ax.plot( fft_abs_hanning[idxLine,:], label="hanning"  )
ax.legend()
ax.grid()

# Build RD

**Check that the first ramp is an up ramp**

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot( A_reshaped[0, :] )
title = "first acquisition " + data_date
ax.set_title(title)
ax.grid()
fig.savefig( f"{data_dir}/{title}.png", bbox_inches='tight')

**Set the shifted flag properly**

In [ ]:
upOrDown = "Up"
#upOrDown = "Down"

if upOrDown == "Down":
    rampDownFirst = 1
    withHanning = 1
    RDc = build_rd_from_data_rampDown( params, A_reshaped, rampDownFirst, withHanning )
elif upOrDown == "Up":
    rampUpFirst = 0
    withHanning = 1
    RDc = build_rd_from_data_rampUp( params, A_reshaped, rampUpFirst, withHanning )

## Save data

In [ ]:
if withHanning:
    hanning = "_hanning"
else:
    hanning = ""

In [ ]:
coupling_RD = np.average(RDc, 0)
coupling_name = f'{data_dir}/coupling_RD_files_{firstFile}_{lastFile}_ramp{upOrDown}{hanning}.npy'
print( f"load {coupling_name}")
np.save( coupling_name, coupling_RD )

In [ ]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [ ]:
RD1_name = f'{data_dir}/RD_files_{firstFile}_{lastFile}_ramp{upOrDown}{hanning}.npy'
print( f"load {RD1_name}")
np.save( RD1_name, RDc) 